## Imports

In [88]:
import io
import re
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from pathlib import Path
from plotly.subplots import make_subplots

pd.set_option('display.max_rows', 4)

d_root = Path("/home/fpavlov/projects/article_conserved_miRNA")
d_data = d_root / "data"
d_img = d_root / "img"

# Input
f_flipon_to_data = d_data / "flipon_to_data.tsv"
f_flipon_to_data_og = d_data / "flipon_to_data_og.tsv"

## Data

In [2]:
"""Initialize generated data for pie charts."""

data_list = [        
('<b>Z-DNA</b> (25,057)',
"""\
9    Promoter (<=1kb) 40.95135480
10   Promoter (1-2kb)  5.89808053
11   Promoter (2-3kb)  3.73518496
4              5' UTR  0.02394349
3              3' UTR  0.81806936
1            1st Exon  0.01596233
7          Other Exon  3.09669181
2          1st Intron  4.93235963
8        Other Intron 12.46258829
6  Downstream (<=300)  0.05586815
5   Distal Intergenic 28.00989664
"""),
('<b>SIDD</b> (15,294)',
"""\
9    Promoter (<=1kb)  6.74032427
10   Promoter (1-2kb)  4.19717573
11   Promoter (2-3kb)  3.37996862
4              5' UTR  0.01307531
3              3' UTR  0.92180962
1            1st Exon  0.04576360
7          Other Exon  0.47724895
2          1st Intron  7.38755230
8        Other Intron 18.59309623
6  Downstream (<=300)  0.07191423
5   Distal Intergenic 58.17207113
"""),
('<b>G4</b> (20,251)',
"""\
9    Promoter (<=1kb) 36.5871723
10   Promoter (1-2kb)  5.5448576
11   Promoter (2-3kb)  3.4760282
4              5' UTR  0.0493754
3              3' UTR  1.1603219
1            1st Exon  0.0246877
7          Other Exon  1.5256999
2          1st Intron  5.4510443
8        Other Intron 13.5930479
6  Downstream (<=300)  0.0740631
5   Distal Intergenic 32.5137017
"""),
('<b>H-DNA</b> (17,098)',
"""\
9    Promoter (<=1kb) 11.690058480
10   Promoter (1-2kb)  4.520467836
11   Promoter (2-3kb)  3.690058480
4              5' UTR  0.005847953
3              3' UTR  0.543859649
1            1st Exon  0.023391813
7          Other Exon  0.345029240
2          1st Intron  8.000000000
8        Other Intron 20.087719298
6  Downstream (<=300)  0.040935673
5   Distal Intergenic 51.052631579
"""),
('<b>Z-DNA & miRNA</b> (3,391)',
"""\
7  Promoter (<=1kb) 26.01002654
8  Promoter (1-2kb)  5.69153642
9  Promoter (2-3kb)  4.30551460
3            5' UTR  0.05897965
2            3' UTR  0.70775582
5        Other Exon  1.12061339
1        1st Intron  7.01857859
6      Other Intron 18.43114126
4 Distal Intergenic 36.65585373
"""),
('<b>SIDD & miRNA</b> (12,143)',
"""\
9    Promoter (<=1kb)  6.349337067
10   Promoter (1-2kb)  4.315243350
11   Promoter (2-3kb)  3.434077246
4              5' UTR  0.008235197
3              3' UTR  0.897636498
1            1st Exon  0.041175986
7          Other Exon  0.502347031
2          1st Intron  7.502264679
8        Other Intron 18.809190480
6  Downstream (<=300)  0.074116775
5   Distal Intergenic 58.066375690
"""),
('<b>G4 & miRNA</b> (4,569)',
"""\
8    Promoter (<=1kb) 38.82687678
9    Promoter (1-2kb)  5.66863646
10   Promoter (2-3kb)  3.83015977
3              5' UTR  0.04377325
2              3' UTR  1.24753775
6          Other Exon  2.01356971
1          1st Intron  4.88071788
7        Other Intron 12.54103743
5  Downstream (<=300)  0.02188663
4   Distal Intergenic 30.92580433
"""),
('<b>H-DNA & miRNA</b> (1,474)',
"""\
7   Promoter (<=1kb) 10.71913161
8   Promoter (1-2kb)  4.00271370
9   Promoter (2-3kb)  3.86702849
2             3' UTR  0.81411126
5         Other Exon  0.13568521
1         1st Intron  7.39484396
6       Other Intron 21.70963365
4 Downstream (<=300)  0.06784261
3  Distal Intergenic 51.28900950
"""),
('<b>Z-DNA & SIDD(±100p)</b> (757)',
"""\
8    Promoter (<=1kb) 20.0527704
9    Promoter (1-2kb)  6.0686016
10   Promoter (2-3kb)  4.0897098
3              3' UTR  0.9234828
1            1st Exon  0.1319261
6          Other Exon  0.9234828
2          1st Intron  6.5963061
7        Other Intron 19.3931398
5  Downstream (<=300)  0.1319261
4   Distal Intergenic 41.6886544
"""),
('<b>SIDD(±100bp)</b> (15,294)',
"""\
9    Promoter (<=1kb)  7.20449791
10   Promoter (1-2kb)  4.11218619
11   Promoter (2-3kb)  3.47803347
4              5' UTR  0.02615063
3              3' UTR  0.96103556
1            1st Exon  0.05230126
7          Other Exon  0.84989540
2          1st Intron  7.29602510
8        Other Intron 18.05700837
6  Downstream (<=300)  0.05883891
5   Distal Intergenic 57.90402720
"""),
('<b>G4 & SIDD(±100bp)</b> (2,250)',
"""\
6  Promoter (<=1kb)  6.7555556
7  Promoter (1-2kb)  4.4888889
8  Promoter (2-3kb)  3.3333333
2            3' UTR  0.7555556
4        Other Exon  0.1777778
1        1st Intron  8.4444444
5      Other Intron 19.8222222
3 Distal Intergenic 56.2222222
"""),
('<b>H-DNA & SIDD(±100bp)</b> (2,016)',
"""\
7   Promoter (<=1kb) 10.16865079
8   Promoter (1-2kb)  4.76190476
9   Promoter (2-3kb)  2.77777778
2             3' UTR  0.99206349
5         Other Exon  0.24801587
1         1st Intron  7.44047619
6       Other Intron 20.08928571
4 Downstream (<=300)  0.09920635
3  Distal Intergenic 53.42261905
"""),
('<b>Z-DNA & SIDD(±200bp)</b> (1,116)',
"""\
9    Promoter (<=1kb) 23.63473590
10   Promoter (1-2kb)  6.26678603
11   Promoter (2-3kb)  3.67054611
4              5' UTR  0.08952551
3              3' UTR  0.98478066
1            1st Exon  0.08952551
7          Other Exon  1.07430618
2          1st Intron  6.17726052
8        Other Intron 18.53178156
6  Downstream (<=300)  0.08952551
5   Distal Intergenic 39.39122650
"""),
('<b>SIDD(±200bp)</b> (15,294)',
"""\
9    Promoter (<=1kb)  7.64252092
10   Promoter (1-2kb)  3.98797071
11   Promoter (2-3kb)  3.53687238
4              5' UTR  0.03268828
3              3' UTR  1.01987448
1            1st Exon  0.05230126
7          Other Exon  1.26830544
2          1st Intron  7.21757322
8        Other Intron 17.54053347
6  Downstream (<=300)  0.06537657
5   Distal Intergenic 57.63598326
"""),
('<b>G4 & SIDD(±200bp)</b> (2,534)',
"""\
7   Promoter (<=1kb)  9.5106551
8   Promoter (1-2kb)  4.8539858
9   Promoter (2-3kb)  3.5911602
2             3' UTR  1.0260458
5         Other Exon  0.1973165
1         1st Intron  7.8926598
6       Other Intron 19.1002368
4 Downstream (<=300)  0.0394633
3  Distal Intergenic 53.7884767
"""),
('<b>H-DNA & SIDD(±200bp)</b> (2,249)',
"""\
7   Promoter (<=1kb) 10.04891063
8   Promoter (1-2kb)  4.89106269
9   Promoter (2-3kb)  2.97910182
2             3' UTR  0.97821254
5         Other Exon  0.22232103
1         1st Intron  7.87016452
6       Other Intron 20.36460649
4 Downstream (<=300)  0.08892841
3  Distal Intergenic 52.55669186
"""),
('<b>Z-DNA & SIDD(±500bp)</b> (1,801)',
"""\
9    Promoter (<=1kb) 33.0743618
10   Promoter (1-2kb)  6.6037736
11   Promoter (2-3kb)  3.8845727
4              5' UTR  0.0554939
3              3' UTR  1.4983352
1            1st Exon  0.1109878
7          Other Exon  1.8867925
2          1st Intron  4.8834628
8        Other Intron 15.5937847
6  Downstream (<=300)  0.0554939
5   Distal Intergenic 32.3529412
"""),
('<b>SIDD(±500bp)</b> (15,294)',
"""\
9    Promoter (<=1kb)  8.96312762
10   Promoter (1-2kb)  3.64147490
11   Promoter (2-3kb)  3.56302301
4              5' UTR  0.07845188
3              3' UTR  1.23561715
1            1st Exon  0.03922594
7          Other Exon  1.96129707
2          1st Intron  7.00183054
8        Other Intron 16.46835774
6  Downstream (<=300)  0.06537657
5   Distal Intergenic 56.98221757
"""),
('<b>G4 & SIDD(±500bp)</b> (3,089)',
"""\
7   Promoter (<=1kb) 15.31239883
8   Promoter (1-2kb)  5.05017805
9   Promoter (2-3kb)  3.59339592
2             3' UTR  1.23017158
5         Other Exon  0.48559404
1         1st Intron  7.08967303
6       Other Intron 18.06409841
4 Downstream (<=300)  0.09711881
3  Distal Intergenic 49.07737132
"""),
('<b>H-DNA & SIDD(±500bp)</b> (2,560)',
"""\
7   Promoter (<=1kb) 11.1718750
8   Promoter (1-2kb)  4.6875000
9   Promoter (2-3kb)  3.0468750
2             3' UTR  0.9765625
5         Other Exon  0.2343750
1         1st Intron  7.8515625
6       Other Intron 20.1562500
4 Downstream (<=300)  0.0781250
3  Distal Intergenic 51.7968750
"""),
]

data_list = np.array(data_list)

print([f"{i}: {x[0]}" for i, x in enumerate(data_list)])
len(data_list)

['0: <b>Z-DNA</b> (25,057)', '1: <b>SIDD</b> (15,294)', '2: <b>G4</b> (20,251)', '3: <b>H-DNA</b> (17,098)', '4: <b>Z-DNA & miRNA</b> (3,391)', '5: <b>SIDD & miRNA</b> (12,143)', '6: <b>G4 & miRNA</b> (4,569)', '7: <b>H-DNA & miRNA</b> (1,474)', '8: <b>Z-DNA & SIDD(±100p)</b> (757)', '9: <b>SIDD(±100bp)</b> (15,294)', '10: <b>G4 & SIDD(±100bp)</b> (2,250)', '11: <b>H-DNA & SIDD(±100bp)</b> (2,016)', '12: <b>Z-DNA & SIDD(±200bp)</b> (1,116)', '13: <b>SIDD(±200bp)</b> (15,294)', '14: <b>G4 & SIDD(±200bp)</b> (2,534)', '15: <b>H-DNA & SIDD(±200bp)</b> (2,249)', '16: <b>Z-DNA & SIDD(±500bp)</b> (1,801)', '17: <b>SIDD(±500bp)</b> (15,294)', '18: <b>G4 & SIDD(±500bp)</b> (3,089)', '19: <b>H-DNA & SIDD(±500bp)</b> (2,560)']


20

In [85]:
plotly_df = pd.DataFrame()

for data_name, data_text in data_list:
    df = pd.read_csv(
        io.StringIO(re.sub(" +", " ", data_text)),
        sep=" ",
        header=None,
        names=["idx", "anno1", "anno2", "percentage"],
    ).assign(
        percentage=lambda x: x["percentage"] / 100,
        annotation=lambda x: x["anno1"] + " " + x["anno2"],
        flipon=data_name,
        group=lambda x: data_name.split(' ')[0].split('(')[0].replace('</b>', '')
    )[
        ['group', "flipon", "annotation", "percentage"]
    ]

    plotly_df = pd.concat([plotly_df, df], ignore_index=True)

plotly_df


,group,flipon,annotation,percentage
0,<b>Z-DNA,"<b>Z-DNA</b> (25,057)",Promoter (<=1kb),0.409514
1,<b>Z-DNA,"<b>Z-DNA</b> (25,057)",Promoter (1-2kb),0.058981
2,<b>Z-DNA,"<b>Z-DNA</b> (25,057)",Promoter (2-3kb),0.037352
3,<b>Z-DNA,"<b>Z-DNA</b> (25,057)",5' UTR,0.000239
4,<b>Z-DNA,"<b>Z-DNA</b> (25,057)",3' UTR,0.008181
...,...,...,...,...
196,<b>H-DNA,"<b>H-DNA & SIDD(±500bp)</b> (2,560)",Other Exon,0.002344
197,<b>H-DNA,"<b>H-DNA & SIDD(±500bp)</b> (2,560)",1st Intron,0.078516
198,<b>H-DNA,"<b>H-DNA & SIDD(±500bp)</b> (2,560)",Other Intron,0.201563
199,<b>H-DNA,"<b>H-DNA & SIDD(±500bp)</b> (2,560)",Downstream (<=300),0.000781


In [86]:
flipon_order = [
    "<b>Z-DNA</b> (25,057)",
    "<b>Z-DNA & miRNA</b> (3,391)",
    "<b>Z-DNA & SIDD(±100p)</b> (757)",
    "<b>Z-DNA & SIDD(±200bp)</b> (1,116)",
    "<b>Z-DNA & SIDD(±500bp)</b> (1,801)",
    "<b>G4</b> (20,251)",
    "<b>G4 & miRNA</b> (4,569)",
    "<b>G4 & SIDD(±100bp)</b> (2,250)",
    "<b>G4 & SIDD(±200bp)</b> (2,534)",
    "<b>G4 & SIDD(±500bp)</b> (3,089)",
    "<b>H-DNA</b> (17,098)",
    "<b>H-DNA & miRNA</b> (1,474)",
    "<b>H-DNA & SIDD(±100bp)</b> (2,016)",
    "<b>H-DNA & SIDD(±200bp)</b> (2,249)",
    "<b>H-DNA & SIDD(±500bp)</b> (2,560)",
    "<b>SIDD</b> (15,294)",
    "<b>SIDD & miRNA</b> (12,143)",
    "<b>SIDD(±100bp)</b> (15,294)",
    "<b>SIDD(±200bp)</b> (15,294)",
    "<b>SIDD(±500bp)</b> (15,294)",
]


In [95]:
flipon_to_data = pd.read_table(f_flipon_to_data_og)

temp = flipon_to_data[
    (flipon_to_data['group'] == 'z-dna')
    & (flipon_to_data['annotation']=="Promoter (<=1kb)")
    # & (~flipon_to_data['cCRE (+-200bp slop)'].isna())
]

temp.shape[0] / flipon_to_data[flipon_to_data['group']=='z-dna'].shape[0]

/tmp/ipykernel_2542967/867180507.py:1: DtypeWarning:

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.



0.4095462345851459

In [107]:
import plotly.express as px

fig = px.bar(
    plotly_df,
    x="percentage",
    facet_row="group",
    color="annotation",
    y="flipon",
    text_auto=".1%",
    orientation="h",
    category_orders={"flipon": flipon_order},
)


# fig.update_yaxes(
#     ticks="",
#     type="category",
#     categoryarray=plotly_df.sort_values("annotation", ascending=True)["annotation"].unique(),
#     categoryorder="array",
# )

fig.update_xaxes(
    dtick=.1,
    tickformat='.0%',
)

fig.update_yaxes(
    matches=None,
)

fig.update_layout(
    font=dict(family="monospace", size=12),
    uniformtext_minsize=12,
    uniformtext_mode="hide",
    height=900,
    width=1400,
    template="ggplot2",
)

fig.show()


## Plots

In [11]:
"""Define necessary functions for plotting."""


def get_category_value(category: str, raw_text_data: str):
    """Parse raw text input from chipSeeker and return value."""

    if category not in raw_text_data:
        return 0
    return float(raw_text_data.split(category)[1].split("\n")[0].strip())


def draw_pie_subplots(data: list, rows: int, cols: int, title: str = "result"):
    """Draw a pie chart."""

    fig = make_subplots(
        rows=rows,
        cols=cols,
        specs=np.full((rows, cols), {"type": "domain"}).tolist(),
        horizontal_spacing=0.04,
        subplot_titles=[x[0] for x in data],
    )

    for i, (data_name, data_raw) in enumerate(data):
        categories = pd.DataFrame(
            {
                "label": [
                    "Promoter (<=1kb)",
                    "Promoter (1-2kb)",
                    "Promoter (2-3kb)",
                    "5' UTR",
                    "3' UTR",
                    "1st Exon",
                    "Other Exon",
                    "1st Intron",
                    "Other Intron",
                    "Downstream (<=300)",
                    "Distal Intergenic",
                ],
                "parent": [
                    "Promoter",
                    "Promoter",
                    "Promoter",
                    "Gene body",
                    "Gene body",
                    "Gene body",
                    "Gene body",
                    "Gene body",
                    "Gene body",
                    "Intergenic",
                    "Intergenic",
                ],
                "value": None,
            }
        )

        for idx, row in categories.iterrows():
            categories.loc[idx, "value"] = get_category_value(row["label"], data_raw)

        # categories.loc[len(categories)] = ['Total', '', categories['value'].sum()]
        # categories.loc[len(categories)] = ['Promoter', '', categories[categories['parent']=="Promoter"]['value'].sum()]
        # categories.loc[len(categories)] = ['Gene body', '', categories[categories['parent']=="Gene body"]['value'].sum()]
        # categories.loc[len(categories)] = ['Intergenic', '', categories[categories['parent']=="Intergenic"]['value'].sum()]

        # fig.add_trace(
        #     go.Sunburst(
        #         sort=False,
        #         rotation=0,
        #         labels=categories['label'],
        #         parents=categories['parent'],
        #         values=categories['value'],
        #         name=data_name,
        #         branchvalues='total',
        #         insidetextorientation='radial',
        #     ),
        #     1,
        #     i + 1,
        # )
        fig.add_trace(
            go.Pie(
                sort=False,
                rotation=0,
                direction="clockwise",
                labels=categories["label"],
                values=categories["value"],
                name=data_name,
                # marker=dict(colors=['royalblue', 'cornflowerblue', 'dodgerblue', 'tomato', 'orangered', 'darkorange', 'orange', 'coral', 'limegreen', 'black', 'seagreen']),
                # marker=dict(colors=px.colors.sequential.thermal),
            ),
            1,
            i + 1,
        )

    fig.update_traces(
        hole=0.35, hoverinfo="label+percent+name", texttemplate="%{value:.2f}%"
    )

    fig.update_annotations(yshift=20)

    fig.update_layout(
        uniformtext_minsize=10,
        uniformtext_mode="hide",
        font=dict(family="monospace", size=14),
        margin=dict(t=60, b=0, l=0),
        legend=dict(
            # yanchor="top",
            # y=0.99,
            xanchor="left",
            x=1.025
        ),
        height=500,
        width=1800,
    )
    fig.write_image(title + ".png")
    fig.show()


In [12]:
"""Draw the figures."""

figure_names = [
    'flipons', 'flipons_and_mirna', 'flipons_100bp', 'flipons_200bp', 'flipons_500bp', 
]
for i in range(len(figure_names)):
    draw_pie_subplots(
        data_list[i * 4 : (i * 4) + 4], 1, 4, f"../img/mm10_kouzine_ssDNA_pie_chart_{i}_{figure_names[i]}"
    )


In [ ]:
def draw_sunburst(data: list, rows: int, cols: int, title: str = "result"):
    fig = make_subplots(
        rows=rows,
        cols=cols,
        specs=np.full((rows, cols), {"type": "domain"}).tolist(),
        horizontal_spacing=0.04,
        subplot_titles=[x[0] for x in data],
    )

    for i, (data_name, data_raw) in enumerate(data):
        categories = pd.DataFrame(
            {
                "label": [
                    "Promoter (<=1kb)",
                    "Promoter (1-2kb)",
                    "Promoter (2-3kb)",
                    "5' UTR",
                    "3' UTR",
                    "1st Exon",
                    "Other Exon",
                    "1st Intron",
                    "Other Intron",
                    "Downstream (<=300)",
                    "Distal Intergenic",
                ],
                "parent": [
                    "Promoter",
                    "Promoter",
                    "Promoter",
                    "Gene body",
                    "Gene body",
                    "Gene body",
                    "Gene body",
                    "Gene body",
                    "Gene body",
                    "Intergenic",
                    "Intergenic",
                ],
                "value": None,
            }
        )

        for idx, row in categories.iterrows():
            categories.loc[idx, "value"] = get_category_value(row["label"], data_raw)

        categories.loc[len(categories)] = ['Total', '', categories['value'].sum()]
        categories.loc[len(categories)] = ['Promoter', '', categories[categories['parent']=="Promoter"]['value'].sum()]
        categories.loc[len(categories)] = ['Gene body', '', categories[categories['parent']=="Gene body"]['value'].sum()]
        categories.loc[len(categories)] = ['Intergenic', '', categories[categories['parent']=="Intergenic"]['value'].sum()]

        fig.add_trace(
            go.Sunburst(
                sort=False,
                rotation=0,
                labels=categories['label'],
                parents=categories['parent'],
                values=categories['value'],
                name=data_name,
                branchvalues='total',
                insidetextorientation='radial',
            ),
            1,
            i + 1,
        )

    fig.update_traces(
        hole=0.35, hoverinfo="label+percent+name", texttemplate="%{value:.2f}%"
    )

    fig.update_annotations(yshift=20)

    fig.update_layout(
        uniformtext_minsize=10,
        uniformtext_mode="hide",
        font=dict(family="monospace", size=14),
        margin=dict(t=60, b=0, l=0),
        legend=dict(
            # yanchor="top",
            # y=0.99,
            xanchor="left",
            x=1.025
        ),
        height=500,
        width=1800,
    )
    fig.write_image(title + ".png")
    fig.show()
